In [6]:
import pandas as pd
import numpy as np
import requests
import json
from datetime import datetime
import time

In [7]:
date_start = datetime.strptime("2022-08-23", "%Y-%m-%d") #Указать начальную дату
base_url = "http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities.json"
response = requests.get(base_url) #Базовый адрес и дата
result = json.loads(response.text)
col_name = result['history']['columns'] #Задаем имена колонок
data = pd.DataFrame(columns = col_name)

In [8]:
nwd_url = 'https://raw.githubusercontent.com/d10xa/holidays-calendar/master/json/calendar.json' #Ссылка на календарь праздников
nwd_response = requests.get(nwd_url)
nwd_result = json.loads(nwd_response.text)

In [9]:
no_work_day = pd.DataFrame(nwd_result['holidays'], columns = ['holidays'])
no_work_day['holidays_date'] = pd.to_datetime(no_work_day['holidays'], format="%Y-%m-%d")
no_work_day = no_work_day[no_work_day['holidays_date'] >= date_start]
datelist = pd.date_range(start="2022-08-23",end="2023-09-29")
datelist = pd.DataFrame(list(datelist.to_pydatetime()), columns = ['holidays_date'])
day_list = datelist["holidays_date"].values #Список дат в диапазоне
nwd_list = no_work_day["holidays_date"].values #Список праздничныйх дней
traide_day = []
for i in day_list:
    if i not in nwd_list:
        traide_day.append(i)

In [10]:
for i in traide_day:
    url_date = np.datetime_as_string(i, unit='D') #Передаем дату в виде строки формата ГГГГ-ММ-ДД
    url_one_page = base_url + '?date=' + url_date
    response = requests.get(url_one_page) #Базовый адрес и дата
    result = json.loads(response.text)
    total_line = result['history.cursor']['data'][0][1]#Узнаем количество строк, если их 0 (нет торгов), то берем следующий день
    if total_line == 0:
        continue
    step = total_line//100 #Узнаем количество сотен
    ost = total_line%100 #Узнаем количество едениц
    resp_date = result['history']['data'] #Извлекаем полученные на 1 странице данные 
    data_p_one = pd.DataFrame(resp_date, columns = col_name) #Полеченные данные записываем в датафрейм
    data = pd.concat([data, data_p_one], ignore_index=True) #Записываем в изначальный датафрейм полученные данные
    #Цикл загрузки остальных строк
    for j in range(1,step+1):
        page_line = '&start=' + str(j*100)
        url_next_page = url_one_page + page_line
        response = requests.get(url_next_page)
        result = json.loads(response.text)
        resp_date = result['history']['data']
        data_next_page = pd.DataFrame(resp_date, columns = col_name)
        data = pd.concat([data, data_next_page], ignore_index=True)
        time.sleep(3)

In [11]:
#data_cap_q1 = pd.read_html('https://www.moex.com/a8135')
#data_cap_q2 = pd.read_html('https://www.moex.com/a8177')
#cap_q1 = data_cap_q1[0]
#cap_q2 = data_cap_q2[0]

In [12]:
data.to_csv('trade_data_14102023.csv', encoding='cp1251', sep=';')
#cap_q1.to_csv('cap_q1.csv', encoding='cp1251')
#cap_q2.to_csv('cap_q2.csv', encoding='cp1251')

In [13]:
data

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,...,MARKETPRICE2,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR
0,TQBR,2022-08-23,АбрауДюрсо,ABRD,235,2.247400e+06,182.00,180.000,182.00,181.00,...,181.000,181.000,181.0,2.247400e+06,2.247400e+06,2247400.0,0,3,SUR,0.56
1,TQBR,2022-08-23,АСКО ао,ACKO,0,0.000000e+00,NaN,NaN,NaN,3.58,...,NaN,NaN,3.58,0.000000e+00,0.000000e+00,0.0,0,3,SUR,NaN
2,TQBR,2022-08-23,Система ао,AFKS,12743,3.430188e+08,14.45,14.337,14.88,14.68,...,14.598,14.598,14.68,3.430188e+08,3.430188e+08,343018836.6,0,3,SUR,1.66
3,TQBR,2022-08-23,Аэрофлот,AFLT,7082,1.333748e+08,27.06,26.960,27.22,27.18,...,27.100,27.100,27.18,1.333748e+08,1.333748e+08,133374843.0,0,3,SUR,0.67
4,TQBR,2022-08-23,AGRO-гдр,AGRO,3473,4.965677e+07,892.80,885.000,910.00,886.00,...,897.400,897.400,886.0,4.965677e+07,4.965677e+07,49656770.8,0,3,SUR,-0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69385,TQBR,2023-09-29,Yandex clA,YNDX,36091,2.175166e+09,2448.80,2392.000,2449.40,2415.20,...,2421.000,2421.000,None,1.957770e+09,1.957770e+09,None,0,3,SUR,-1.52
69386,TQBR,2023-09-29,ТНСэнЯр,YRSB,32,5.299400e+05,1346.00,1302.000,1348.00,1302.00,...,1324.000,1324.000,None,5.299400e+05,5.299400e+05,None,0,3,SUR,-1.81
69387,TQBR,2023-09-29,ТНСэнЯр-п,YRSBP,50,5.682600e+05,366.50,358.500,366.50,358.50,...,364.500,364.500,None,5.682600e+05,5.682600e+05,None,0,3,SUR,-2.45
69388,TQBR,2023-09-29,ЗИЛ ао,ZILL,185,2.295960e+06,3585.00,3555.000,3655.00,3590.00,...,3620.000,3620.000,None,2.295960e+06,2.295960e+06,None,0,3,SUR,0.56
